Part 1

In [149]:
import pandas as pd
import re

In [150]:
def str_int(s):
    return list(map(int, s.strip().split(' ')))

In [158]:
# file = "test.dat"
file = "day.dat"
lines = open(file,'r').read().split('\n')
seeds = pd.DataFrame({'seed':str_int(lines[0].split(':')[1])})
map_types = ['seed-to-soil map:','soil-to-fertilizer map:',
             'fertilizer-to-water map:','water-to-light map:',
             'light-to-temperature map:','temperature-to-humidity map:',
             'humidity-to-location map:']

df = pd.DataFrame(columns=['map_from','map_to','destination','source','length'])
for l in lines[1:]:
    if l in map_types:
        match = re.match(r'^(\w+)\-to\-(\w+)', l)
        map_from, map_to = match.group(1), match.group(2)
    elif l != '':
        dest,src,length = str_int(l)
        df = pd.concat([df,pd.DataFrame({'map_from':map_from, 'map_to':map_to,
                                         'destination':dest, 'source':src,
                                         'length':length}, index=[0])])
df = df.reset_index(drop=True)   
df.head()

,map_from,map_to,destination,source,length
0,seed,soil,2211745924,1281207339,39747980
1,seed,soil,3648083739,2564129012,145170114
2,seed,soil,4171944574,2333022880,44675857
3,seed,soil,540694760,848661020,78793182
4,seed,soil,256996824,588160543,260500477


In [160]:
seeds.head()

,seed
0,3082872446
1,316680412
2,2769223903
3,74043323
4,4131958457


In [161]:

from_col = 'seed'
map_list = df[df['map_from']==from_col]
to_col = list(map_list['map_to'])[0]
current_col = seeds[from_col]
seeds[to_col] = current_col
while current_col is not None:
    for i, s in enumerate(current_col):
        for _, m in map_list.iterrows():
            if s in range(m['source'], m['source']+m['length']):
                seeds.at[i, to_col] = m['destination'] + s - m['source']
    from_col = to_col
    if from_col == 'location':
        break
    map_list = df[df['map_from']==from_col]
    to_col = list(map_list['map_to'])[0]
    current_col = seeds[from_col]
    seeds[to_col] = current_col
 
seeds.head()

,seed,soil,fertilizer,water,light,temperature,humidity,location
0,3082872446,3146600596,3070093545,1126696133,2291994928,1972007158,1311845980,1085436433
1,316680412,529355900,2791332414,847935002,1268090341,64848467,84470622,84470622
2,2769223903,4276545208,4187659549,2037385377,2943142680,1678956429,1018795251,797022648
3,74043323,2119458269,1459707245,4064254365,3341541755,2961582857,4109588370,3040934752
4,4131958457,2994950048,2994950048,1051552636,2216851431,2473940599,2473940599,2473940599


In [162]:
#answer is 84470622
seeds['location'].min()

84470622
